In [2]:
import numpy as np
import cv2 
import matplotlib
from matplotlib import pyplot as plt

In [3]:
# If we need to check shape of the image
# img = cv2.imread('Dataset/Due_Lipa/293d1b64633207b19316514969b45c8e.jpeg', cv2.IMREAD_UNCHANGED)
# img.shape

In [4]:
# When need to open the image from a window
# cv2.imshow('Original Image', img)
# cv2.waitKey(0)  # Wait for a key press
# cv2.destroyAllWindows()

In [5]:
#if need to change image color into gray
# gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
# gray_img.shape
# plt.imshow(gray_img, cmap='gray')

In [6]:
face_cascade = cv2.CascadeClassifier('./opencv/haarcascade_frontalface_default.xml')
eye_cascade = cv2.CascadeClassifier('./opencv/haarcascade_eye.xml')

# **** To detect face from one image ****
# faces = face_cascade.detectMultiScale(
#     img, scaleFactor=1.05, minNeighbors=2, minSize=(100, 100)
# )
# # faces

In [7]:
# (x,y,w,h) = faces[0]
# x,y,w,h

In [8]:
# **** To draw a rectangle around the face ****
# face_img = cv2.rectangle(img,(x,y),(x+w,y+h),(255,0,0),2)
# plt.imshow(face_img)

In [9]:
# **** To draw rectangles around the face and eyes ****
# cv2.destroyAllWindows()
# for (x,y,w,h) in faces:
#     face_img = cv2.rectangle(img,(x,y),(x+w,y+h),(255,0,0),2)
#     roi_img = img[y:y+h, x:x+w]     # roi = region of interest
#     roi_color = face_img[y:y+h, x:x+w]
#     eyes = eye_cascade.detectMultiScale(roi_img)
#     for (ex,ey,ew,eh) in eyes:
#         cv2.rectangle(roi_color,(ex,ey),(ex+ew,ey+eh),(0,255,0),2)
#         
# plt.figure()
# plt.imshow(face_img, cmap='gray')
# plt.show

In [10]:
# **** to get only the crop image ****
# %matplotlib inline
# plt.imshow(roi_color, cmap='gray')

In [16]:
# A function for get cropped image from face identified images
def get_cropped_image(img_path):
    img = cv2.imread(img_path)
    faces = face_cascade.detectMultiScale(img, scaleFactor=1.05, minNeighbors=2, minSize=(100, 100))
    for (x,y,w,h) in faces:
    # face_img = cv2.rectangle(img,(x,y),(x+w,y+h),(255,0,0),2)
        roi_img = img[y:y+h, x:x+w]     # roi = region of interest
        roi_color = img[y:y+h, x:x+w]
        eyes = eye_cascade.detectMultiScale(roi_img)
        if len(eyes)>=2:
            return roi_color
        # cv2.rectangle(roi_color,(ex,ey),(ex+ew,ey+eh),(0,255,0),2)

In [17]:
# cropped_img = get_cropped_image('./Dataset/Lionel Messi/images.jpeg')
# plt.imshow(cropped_img)

In [18]:
path_to_data = "./Dataset/"
path_to_cropped_data = "./Dataset/cropped/"

In [19]:
import os
img_dirs=[]
for entry in os.scandir(path_to_data):
    if entry.is_dir():
        img_dirs.append(entry.path)

In [20]:
print(img_dirs)

['./Dataset/cropped', './Dataset/Due_Lipa', './Dataset/Lionel_Messi', './Dataset/Ronaldo', './Dataset/Selena_Gomez']


In [21]:
if not os.path.exists(path_to_cropped_data):
    os.makedirs(path_to_cropped_data)

In [22]:
cropped_img_dirs = []
celebrity_file_name_dict = {}

for img_dir in img_dirs:
    count = 1
    celebrity_name = img_dir.split('/')[-1] #celebrity_name = os.path.basename(img_dir)
    print(celebrity_name)
    
    celebrity_file_name_dict[celebrity_name] = []

    for entry in os.scandir(img_dir):
        roi_color = get_cropped_image(entry.path)
        if roi_color is not None:
            cropped_folder = path_to_cropped_data + celebrity_name
            if not os.path.exists(cropped_folder):
                os.mkdir(cropped_folder)
                cropped_img_dirs.append(cropped_folder)
                print("generating folder: ", cropped_folder)
                
            cropped_file_name = celebrity_name + str(count) + ".png"
            cropped_file_path = cropped_folder + "/" + cropped_file_name
            
            cv2.imwrite(cropped_file_path,roi_color)
            celebrity_file_name_dict[celebrity_name].append(cropped_file_path)
            count +=1

cropped
Due_Lipa
generating folder:  ./Dataset/cropped/Due_Lipa
Lionel_Messi
generating folder:  ./Dataset/cropped/Lionel_Messi
Ronaldo
generating folder:  ./Dataset/cropped/Ronaldo
Selena_Gomez
generating folder:  ./Dataset/cropped/Selena_Gomez
